In [1]:
!pip install qpsolvers

In [1]:
import roboticstoolbox as rtb
import spatialmath as sm
import math as m
import spatialgeometry as sg
import numpy as np
import qpsolvers as qp
import swift
from typing import Tuple
# Launch the simulator Swift
env = swift.Swift()
env.launch(realtime=True)


# Create a Panda robot object
UR10e = rtb.models.UR10()

# Set joint angles to ready configuration
UR10e.q = UR10e.qr
# Set joint angles to ready configuration
UR10e.q = [0.0, -m.pi/2, m.pi/2 , -m.pi/2, -m.pi/2, 0]
# Add the Panda to the simulator
env.add(UR10e)


## Add target
target = sg.Sphere(radius =0.01, pose=sm.SE3(0.5, 0.4, 0.0))
env.add(target)
# Set the desired end-effector pose to the location of target
Tep = UR10e.fkine(UR10e.q)
Tep.A[:3, 3] = target.T[:3, 3] # get only position x y z

#Show axis
# end-effector axes
ee_axes = sg.Axes(0.1)
# This pose can be either a spatialmat SE3 or a 4x4 ndarray
ee_axes.T = UR10e.fkine(UR10e.q)
# goal axes
goal_axes = sg.Axes(0.1)
goal_axes.T = Tep
# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes)
# Number of joint in the UR10e which we are controlling
n = 6

In [2]:
Te = UR10e.fkine(UR10e.q)
print(Te)

   0         0         1         0.688     
   0         1         0         0.1639    
  -1         0         0         0.6471    
   0         0         0         1         



In [3]:

arrived = False

while not arrived:

    # The pose of the UR10e's end-effector
    Te = UR10e.fkine(UR10e.q)

    # Transform from the end-effector to desired pose
    eTep = Te.inv() * Tep

    # Spatial error
    e = np.sum(np.abs(np.r_[eTep.t, eTep.rpy() * np.pi/180]))
    
    # Specify the gain for the p_servo method
    gain = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
    # Calulate the required end-effector spatial velocity for the robot
    # to approach the goal. Gain is set to 1.0
    v, arrived = rtb.p_servo(Te, Tep, gain=gain, threshold=0.001)

    # Gain term (lambda) for control minimisation
    Y = 0.01

    # Quadratic component of objective function
    Q = np.eye(n + 6)

    # Joint velocity component of Q
    Q[:n, :n] *= Y

    # Slack component of Q
    Q[n:, n:] = (1 / e) * np.eye(6)

    # The equality contraints
    Aeq = np.c_[UR10e.jacobe(UR10e.q), np.eye(6)]
    beq = v.reshape((6,))

    # The inequality constraints for joint limit avoidance
    Ain = np.zeros((n + 6, n + 6))
    bin = np.zeros(n + 6)

    # The minimum angle (in radians) in which the joint is allowed to approach
    # to its limit
    ps = 0.05

    # The influence angle (in radians) in which the velocity damper
    # becomes active
    pi = 0.9

    # Form the joint limit velocity damper
    Ain[:n, :n], bin[:n] = UR10e.joint_velocity_damper(ps, pi, n)

    # Linear component of objective function: the manipulability Jacobian
    c = np.r_[-UR10e.jacobm().reshape((n,)), np.zeros(6)]

    # The lower and upper bounds on the joint velocity and slack variable
    UR10e_qdlim = [2.175, 2.175, 2.175,  2.175, 2.61,  2.61]
    lb = -np.r_[UR10e_qdlim, 10 * np.ones(6)]
    ub = np.r_[UR10e_qdlim, 10 * np.ones(6)]

    # Solve for the joint velocities dq
    qd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver='daqp')

    # Apply the joint velocities to the UR10e
    UR10e.qd[:n] = qd[:n]

    # Step the simulator by 50 ms
    env.step(0.05)


More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, usin

More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, usin

More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, usin

More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, usin

More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, using robot.ee_links[0]
More than one end-effector present, usin